<a href="https://colab.research.google.com/github/TriYulianti23/RCDS-introduction-to-AI-assisted-programming/blob/main/2VXH-2RCsg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
import pickle

# Step 1: Upload the file
uploaded = files.upload()  # This will open a file chooser dialog

# Step 2: Get the filename (uploaded is a dict)
filename = list(uploaded.keys())[0]
print(f"Uploaded file: {filename}")

# Step 3: Load the .pkl file
with open(filename, "rb") as f:
    data = pickle.load(f)

print("File loaded successfully!")
# Optionally inspect the content
print(type(data))


Saving cas_integrals2RC.pkl to cas_integrals2RC (1).pkl
Uploaded file: cas_integrals2RC (1).pkl
File loaded successfully!
<class 'dict'>


In [36]:
input_file = "cas_integrals2RC.pkl"

# Add necessary import
import openfermion # Added this line
import openfermion.transforms # Added this for get_fermion_operator
import numpy as standard_np # Import standard numpy
import pennylane as qml
from pennylane import numpy as qml_np # Use qml_np for PennyLane's differentiable numpy
# from pennylane.fermi import FermiWord, FermiSentence # No longer needed for this approach

# Step 1: Load Hamiltonian from .pkl file
with open(input_file, "rb") as f:
    hamiltonian_data = pickle.load(f)

# Check structure
print(f"Hamiltonian data type: {type(hamiltonian_data)}")

# Extract one- and two-electron integrals
h1 = hamiltonian_data['h1']
h2 = hamiltonian_data['h2'] # This h2 is (10,10)

# Determine the number of molecular orbitals (spin orbitals / 2) from h1
n_spatial_orbitals = h1.shape[0] # This is 4
n_qubits = 2 * n_spatial_orbitals # Each spatial orbital has two spin orbitals (alpha and beta)

# --- Start: Custom function to expand compressed h2 integrals ---
# Function to map orbital indices (p,q) to a unique combined index (triangular indexing)
# assuming p >= q for canonical representation.
def get_triangular_idx(p, q, n_orbs):
    if p < q:
        p, q = q, p  # Ensure p >= q
    return p * (p + 1) // 2 + q

# Function to expand a compressed 2-electron integral tensor (N_pairs, N_pairs) to (N,N,N,N)
def expand_h2_from_compressed(h2_compressed, n_spatial_orbitals):
    n_pairs = n_spatial_orbitals * (n_spatial_orbitals + 1) // 2
    if h2_compressed.shape != (n_pairs, n_pairs):
        raise ValueError(f"Compressed h2 shape {h2_compressed.shape} does not match expected "
                         f"({n_pairs},{n_pairs}) for {n_spatial_orbitals} orbitals. "
                         f"Expected total elements: {n_pairs**2}, Got: {h2_compressed.size}")

    # Use standard_np.zeros here to create a standard NumPy array
    h2_4d = standard_np.zeros((n_spatial_orbitals, n_spatial_orbitals, n_spatial_orbitals, n_spatial_orbitals))

    # Iterate through all combinations of 4 orbital indices (p,q,r,s)
    for p in range(n_spatial_orbitals):
        for q in range(n_spatial_orbitals):
            for r in range(n_spatial_orbitals):
                for s in range(n_spatial_orbitals):
                    # Get the combined indices for (p,q) and (r,s) pairs
                    idx_pq = get_triangular_idx(p, q, n_spatial_orbitals)
                    idx_rs = get_triangular_idx(r, s, n_spatial_orbitals)

                    # Assign the value from the compressed h2.
                    # This assumes h2_compressed[idx_pq, idx_rs] directly corresponds to the (pq|rs) integral.
                    h2_4d[p, q, r, s] = h2_compressed[idx_pq, idx_rs]

    return h2_4d

# Expand h2 from (10,10) to (4,4,4,4)
try:
    h2_expanded = expand_h2_from_compressed(h2, n_spatial_orbitals)
    print(f"h2 successfully expanded to shape: {h2_expanded.shape}")
except ValueError as e:
    print(f"Error expanding h2: {e}")
    raise e
# --- End: Custom function to expand compressed h2 integrals ---


# Convert integrals to PennyLane Hamiltonian using Jordan-Wigner transformation
# Ensure h1 and h2_expanded are standard NumPy arrays before passing to OpenFermion
# Create an OpenFermion InteractionOperator
interaction_operator = openfermion.InteractionOperator(
    0.0, # constant (e.g., nuclear repulsion energy)
    standard_np.asarray(h1),  # one-body integrals, ensure standard_np array
    standard_np.asarray(h2_expanded) # two-body integrals, ensure standard_np array
)

# Convert the InteractionOperator to an OpenFermion.FermionOperator
fermion_hamiltonian = openfermion.transforms.get_fermion_operator(interaction_operator)

# Convert the OpenFermion.FermionOperator to an OpenFermion.QubitOperator using Jordan-Wigner transformation
jw_qubit_hamiltonian = openfermion.transforms.jordan_wigner(fermion_hamiltonian)

# Convert the OpenFermion.QubitOperator to a PennyLane.Hamiltonian
pauli_coeffs = []
pauli_ops = []

# Iterate through each term in the OpenFermion.QubitOperator
for term_tuple, coeff in jw_qubit_hamiltonian.terms.items():
    pauli_coeffs.append(coeff)

    # Convert OpenFermion term_tuple to PennyLane Pauli operators
    # term_tuple is a tuple of (qubit_idx, pauli_type) for example: ((0, 'X'), (1, 'Y'))
    pauli_word_ops = []
    for qubit_idx, pauli_type in term_tuple:
        if pauli_type == 'X':
            pauli_word_ops.append(qml.PauliX(qubit_idx))
        elif pauli_type == 'Y':
            pauli_word_ops.append(qml.PauliY(qubit_idx))
        elif pauli_type == 'Z':
            pauli_word_ops.append(qml.PauliZ(qubit_idx))
        elif pauli_type == 'I':
            # OpenFermion can include explicit Identity terms, PennyLane prod handles this
            pauli_word_ops.append(qml.Identity(qubit_idx))

    # If no Pauli operators (e.g., for a pure constant term like OpenFermion's ((), coeff) ),
    # PennyLane Hamiltonian can include Identity(0) as a placeholder for a scalar term.
    # However, qml.prod() is better suited for products of operators.
    # If the term is a pure constant, it will typically be represented as Identity on some qubit in OpenFermion's QubitOperator.
    if not pauli_word_ops: # This case should ideally not happen if Identity terms are properly included.
        # This would imply a pure scalar constant, which might need separate handling for PennyLane Hamiltonian.
        # For now, let's assume OpenFermion QubitOperator always produces some Pauli op or Identity.
        pauli_ops.append(qml.Identity(0)) # Placeholder for Identity
    else:
        pauli_ops.append(qml.prod(*pauli_word_ops)) # Create a product of Pauli operators

hamiltonian = qml.Hamiltonian(pauli_coeffs, pauli_ops)

print("Hamiltonian successfully converted for PennyLane.")

# Step 2: Define quantum device
# n_qubits is already calculated. Use this for the device.
dev = qml.device('default.qubit', wires=n_qubits)

# Step 3: Define the quantum circuit
@qml.qnode(dev)
def quantum_circuit(params):
    # Example parameterized circuit (replace with your FAST-VQE ansatz)
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)

    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])

    # Expectation value of Hamiltonian
    return qml.expval(hamiltonian)

# Step 4: Define cost function for optimization
def cost(params):
    # Ensure the cost function returns a real scalar for the optimizer
    return qml_np.real(quantum_circuit(params))

# Step 5: Run optimization (FAST-VQE)
# Use qml_np for parameters that PennyLane will differentiate
params = qml_np.random.rand(n_qubits)  # Initial parameters
opt = qml.GradientDescentOptimizer(stepsize=0.1)
max_iters = 100

for it in range(max_iters):
    params = opt.step(cost, params)
    if it % 10 == 0:
        print(f"Iteration {it}, Energy = {cost(params)}")

# Step 6: Extract optimized energy
optimized_energy = cost(params)
print(f"Optimized quantum energy from FAST-VQE: {optimized_energy}")

Hamiltonian data type: <class 'dict'>
h2 successfully expanded to shape: (4, 4, 4, 4)
Hamiltonian successfully converted for PennyLane.
Iteration 0, Energy = -0.9743595298860728
Iteration 10, Energy = -2.70655190150565
Iteration 20, Energy = -3.664938626343099
Iteration 30, Energy = -3.803196348486545
Iteration 40, Energy = -3.8208512072291985
Iteration 50, Energy = -3.8239874264252607
Iteration 60, Energy = -3.8247031304969954
Iteration 70, Energy = -3.8248982842932056
Iteration 80, Energy = -3.8249571278332493
Iteration 90, Energy = -3.824975727977206
Optimized quantum energy from FAST-VQE: -3.8249813831602375
